In [1]:
%pylab inline
import pandas as pd
import numpy as np
import json
import math
import copy

Populating the interactive namespace from numpy and matplotlib


### P1 : read csv file( the result of plugins searched and prepared)

In [5]:
yt = pd.read_csv('youtube_searchbychannel_readitems_prepared.csv')
yt = yt.drop(['video_id','video_title'],axis=1)
yt.head()

,video_searched,fetched_date,publishedAt_hour,published_date,description,video_tags,vd_category,commentCount,viewCount,favoriteCount,likeCount,dislikeCount,score3,score1,score2
0,Cashmirino,2017-05-23,19,2016-01-01,Cashmirino’s cashmere pieces are handmade and ...,"[u'cashmere', u'children clothing', u'children...",22,0,52,0,0,0,1.724276,1.7160033436347992,1.724276
1,Armand Basi,2017-05-23,14,2016-04-04,NaN,[Null],22,0,368,0,0,0,2.567026,2.5658478186735176,2.567026
2,Cimalpes,2017-05-23,10,2016-04-06,Cimalpes a le plaisir de vous accueillir dans ...,"[u'lifestyle', u'luxury lifestyle', u'luxury s...",19,0,218,0,0,0,2.340444,2.3384564936046046,2.340444
3,Max Mara,2017-05-23,16,2016-09-08,"Six pairs of super stylish, real-life women sh...","[u'Max Mara', u'maxmara', u'Max Mara Official ...",26,11,25658,0,128,40,7.598316,28.51217712128986,74.507892
4,Swarovski,2017-05-23,11,2016-09-19,Crystaldust: a new must-have bracelet. Discove...,[u'Swarovski'],24,53,521188,0,324,6,9.961267,54.859362411391615,101.221005


It includes the infomation as follows:   

brand searched, searched date, the video published date and hour, video desription, video tags, video catogory, and comments, views, likes, dislikes, favorites for this video, 3 types of scores to evaluate brand fame (notoriété)

after read csv file, the format list is treated as string, so we have to change it

In [6]:
y1 = yt.copy()
y1['tags'] = y1['video_tags']

y1['tags'] = y1['tags'].map(lambda x : x[1:-1])
y1['tags'] = y1['tags'].map(lambda x : x.split(','))

y1.head()

,video_searched,fetched_date,publishedAt_hour,published_date,description,video_tags,vd_category,commentCount,viewCount,favoriteCount,likeCount,dislikeCount,score3,score1,score2,tags
0,Cashmirino,2017-05-23,19,2016-01-01,Cashmirino’s cashmere pieces are handmade and ...,"[u'cashmere', u'children clothing', u'children...",22,0,52,0,0,0,1.724276,1.7160033436347992,1.724276,"[u'cashmere', u'children clothing', u'childr..."
1,Armand Basi,2017-05-23,14,2016-04-04,NaN,[Null],22,0,368,0,0,0,2.567026,2.5658478186735176,2.567026,[Null]
2,Cimalpes,2017-05-23,10,2016-04-06,Cimalpes a le plaisir de vous accueillir dans ...,"[u'lifestyle', u'luxury lifestyle', u'luxury s...",19,0,218,0,0,0,2.340444,2.3384564936046046,2.340444,"[u'lifestyle', u'luxury lifestyle', u'luxury..."
3,Max Mara,2017-05-23,16,2016-09-08,"Six pairs of super stylish, real-life women sh...","[u'Max Mara', u'maxmara', u'Max Mara Official ...",26,11,25658,0,128,40,7.598316,28.51217712128986,74.507892,"[u'Max Mara', u'maxmara', u'Max Mara Officia..."
4,Swarovski,2017-05-23,11,2016-09-19,Crystaldust: a new must-have bracelet. Discove...,[u'Swarovski'],24,53,521188,0,324,6,9.961267,54.859362411391615,101.221005,[u'Swarovski']


the function here realises 2 goals :     
1) fusionner des tags dont marques sont mêmes;
2) réaliser une traduction des tags qui ne sont pas écrit en anglais (Normalisation)  
This step takes a long time, you'd better save the results to be more quick next time;

In [ ]:
import copy
import codecs
from textblob import TextBlob
import textblob

def add_tags(l_tags,l):
    l = [i.replace("Null","") for i in l]
    #l_new = [i.replace(" u'","u'") for i in l]
    l = [i.replace("u'","'") for i in l]
    
    temps = copy.deepcopy(l)
    for k in range(len(l)):
        #if re.search(r'\\',l[k]) != None:
            #print ('before',l[k])
        temps[k] = codecs.decode(l[k],'unicode_escape')
        blob = TextBlob(temps[k])
        if blob.detect_language != 'en' :#and blob.detect_language != 'fr' :
            try:
                trans = str(blob.translate(to='en'))
            except textblob.exceptions.NotTranslated:
                #print (l[k],'this translation does not make changes.')
                pass
            else:
                temps[k] = trans
    #print (temps[:3])
    [l_tags.append(i)  for i in temps]
    return l_tags

In [ ]:
y2 = y1.sort_values(['video_searched'],ascending=True)
y2 = y2.reset_index()
y2 = y2.drop('index',axis=1)


list_tags = add_tags([],y2['tags'][2534])
#list_brand_tags = []
#count = 0

for i in range(2535,len(y2)):
    if y2['video_searched'][i] == y2['video_searched'][i-1]:
        list_tags = add_tags(list_tags,y2['tags'][i])
    else:
        count = count+1
        list_brand_tags.append([y2['video_searched'][i-1], list_tags])
        print (i,count,y2['video_searched'][i-1],list_tags[:3],'\n')
        list_tags = add_tags([],y2['tags'][i])

#translate = translate_tags(list_tags)
list_brand_tags.append([y2['video_searched'][len(y2)-1], list_tags])

     
brand_tags = pd.DataFrame(list_brand_tags, columns=['brand','tags'])
print (brand_tags.shape)
print (brand_tags.iloc[1,1])

In [ ]:
print (brand_tags.shape)
brand_tags.head()

### P2 : text mining process 

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from difflib import SequenceMatcher

def similar (a,b):
    return SequenceMatcher(None,a,b).ratio()

In [ ]:
print (similar('apple','apples'))
print (similar('adidas','Adidas'))

substitute similair spellings with the shortest one 
(to some degree, it avoids the mistake of singular or plural format, etc)

In [ ]:
list_new = []
for i in range(len(brand_tags)):

    #order = df_tags['tags'][i].values.tolist()
    order = brand_tags['tags'][i].copy()
    list_arr = np.zeros(len(order))
    #print (order)
    for j in range(len(order)):
        if list_arr[j] == 0:
            for k in range(j+1,len(order)):
                if similar(order[j],order[k]) == 1 and list_arr[k] != 1:
                    list_arr[k] = 1
                elif similar(order[j],order[k]) > 0.8 and list_arr[k] != 1:
                    #print ("yes,found some word with same meanings!")
                    #print (i,df_tags['brand'][i],df_tags['tags'][i][j],df_tags['tags'][i][k],'\n\n')
                    if len(order[k]) > len(order[j]):
                        order[k] = order[j]
                    else:
                        order[j] = order[k]
                    list_arr[k] = 1
    order = [x.lower()  for x in order]
    print (i,order[:5])
    list_new.append(order)

In [ ]:
brand_tags['sub_tags'] = pd.Series(list_new)
brand_tags.head()

drop certain rows with null tags (it does not make sense in later analyse)

In [ ]:
arr_null = []
for i in range(len(brand_tags)):
    if brand_tags.sub_tags[i] == len(brand_tags.sub_tags[i])*[''] :
        print ("line",i,brand_tags.brand[i],'cannot be validated.')
        arr_null.append(i)
brand_tags = brand_tags.drop(brand_tags.index[arr_null])
brand_tags = brand_tags.reset_index()
brand_tags = brand_tags.drop('index',axis=1)
brand_tags.head()

In [ ]:
brand_tags.to_csv('brand_tags0606.csv',encoding='utf-8')

## tf-idf
tf-idf => svd to reduce the dimensionality => tsne to projection in 2 dimension (for better visualisation the final result)

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans, MiniBatchKMeans
from nltk.corpus import stopwords

stop = set(stopwords.words('english')+stopwords.words('french')+stopwords.words('spanish'))

df_analyse = brand_tags.copy()
df_data = df_analyse.iloc[:,2:3]

df_data['sub_tags'] = [ ' '.join(df_data['sub_tags'][i]) for i in range(len(df_data))]

all_data = df_data.values.tolist()
all_data = [','.join(all_data[i]) for i in range(len(all_data))]
print (len(all_data))


# (the constructor options use_idf=False, norm=None) the same as CountVectorizer
# Tfidf vectorizer:
#   - Strips out “stop words”
#   - Filters out terms that occur in more than half of the docs (max_df=0.5)
#   - Filters out terms that occur in only one document (min_df=2).
#   - Selects the 10,000 most frequently occuring words in the corpus.
#   - Normalizes the vector (L2 norm of 1.0) to normalize the effect of 
#     document length on the tf-idf values. 

#vectorizer = TfidfVectorizer(use_idf=True,stop_words=stop)
vectorizer = TfidfVectorizer(max_df=0.8,min_df=30,use_idf=True,stop_words=stop)
X = vectorizer.fit_transform(all_data)
word = vectorizer.get_feature_names()
print (X.shape)  
print (word)
#print (X.toarray()[0])

In [ ]:
#svd

svd = TruncatedSVD(n_components=20)
lsa = make_pipeline(svd,Normalizer(copy=False))

svd_trans = lsa.fit_transform(X)
#print (svd.explained_variance_ratio_)
explained_variance = svd.explained_variance_ratio_.sum()
print (explained_variance)

In [ ]:
# tsne

import matplotlib as mpl
import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D

from sklearn.manifold import TSNE


t_sne = TSNE(n_components=2,init='pca',perplexity=50,random_state=60,n_iter=5000)
tsne_model = t_sne.fit_transform(svd_trans)
np.set_printoptions(suppress=True)
#print (t_sne_model_3)

print (t_sne.kl_divergence_)
print (t_sne.embedding_)

df_tsne = pd.DataFrame(tsne_model)

fig = plt.figure(figsize=(12,8))

# make the scatter
scat = plt.scatter(df_tsne.iloc[:,0], df_tsne.iloc[:,1])
plt.show()

## KMeans and Visualisation
define the number of kmeans centers : when scores of kmeans do not change so much, we then choose the value with highest silhouette score 

In [ ]:
res = []

center_iner = []
center_score = []

for k in  np.arange(5,55,5):
    km = KMeans(n_clusters=k, n_init=10, random_state=60).fit(svd_trans)
    res.append(metrics.silhouette_score(svd_trans, km.labels_))
    center_iner.append(km.inertia_)
    center_score.append(km.score(svd_trans))

    #print (k/5-2)
print(res)

#graphique
plt.title("Silhouette")
plt.xlabel("# of clusters")
plt.plot(np.arange(10), res)
plt.plot(np.arange(10), center_iner,'-b',label='inertia')
#plt.plot(np.arange(10,100,10), center_score,'-r',label='score')
plt.show()

In [ ]:
center = 25
# pay attention here we use X instead of scale(X)
km3 = KMeans(n_clusters = center, init='k-means++', n_init=10, random_state=60).fit(svd_trans)

In [ ]:
df_visual = df_tsne.copy()
df_visual['label'] = pd.DataFrame(km3.labels_)
print (df_visual.head())

fig = plt.figure(figsize=(12,8))
N = 25
ax = plt.subplot(111)
cmap = plt.cm.jet
cmaplist = [cmap(i) for i in range(cmap.N)]
cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)
bounds = np.linspace(0,N,N+1)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

scat = ax.scatter(df_visual.iloc[:,0],df_visual.iloc[:,1],c=df_visual.label,cmap=cmap,norm=norm)
cb = plt.colorbar(scat, spacing='proportional',ticks=bounds).set_label('Clustering map')

plt.show()

In [ ]:
# list : important words en terms of tags corresponding to th number of clustering
df_visual['brand'] = brand_tags.brand
df_visual.columns = ['x','y','label','brand']
km_index = df_visual.sort_values(by=['label'],ascending=[True])
km_index= km_index.reset_index()
print (km_index.head())


original_space_centroids = svd.inverse_transform(km3.cluster_centers_)
order_centroids = original_space_centroids.argsort()[:, ::-1]


word = []

terms = vectorizer.get_feature_names()
print (len(terms))

for i in range(center):
    arr3 = []
    for ind in order_centroids[i, :3]:
        arr3.append(terms[ind])
    word.append([arr3])
    
key_W = pd.DataFrame(word, columns=['important_tags'])
key_W

#### this csv file is very important because it contains all the necessary infomations for brands ( the brand name, the position information in tsne matrix, the clustering result of kmeans )
then we can create a matrix of similarity for brands using these position informations

In [ ]:
km_index.to_csv('yt_tags_sorted0606.csv',encoding='utf-8')

### P3 :  create a linked network (brand&group, group&tags)

In [ ]:
# calculer each gravity centre for each clustering group

gravity = []

pos_x = [km_index.iloc[0,1]]
pos_y = [km_index.iloc[0,2]]

for i in range(1,len(km_index)):
    if km_index.label[i] == km_index.label[i-1]:
        pos_x.append(km_index.iloc[i,1])
        pos_y.append(km_index.iloc[i,2])
    else:
        #print (i,sum(pos_x),len(pos_x),sum(pos_x)/float(len(pos_x)), sum(pos_y)/float(len(pos_y)))
        gravity.append([sum(pos_x)/float(len(pos_x)), sum(pos_y)/float(len(pos_y))])
        pos_x = [km_index.iloc[i,1]]
        pos_y = [km_index.iloc[i,2]]

        
gravity.append([sum(pos_x)/float(len(pos_x)), sum(pos_y)/float(len(pos_y))])
print (len(gravity))
df_gravity = pd.DataFrame(gravity,columns=['x','y'])
df_gravity['tags'] = key_W.important_tags
print (df_gravity.head())
df_gravity.to_csv('yt_tags_gravity0606.csv',encoding='utf-8')

In [ ]:
df_gravity_new = df_gravity.copy()
#df_gravity_new[['tags1','tags2','tags3','tags4','tags5']] = pd.DataFrame([x for x in df_gravity.tags])
df_gravity_new[['tags1','tags2','tags3']] = pd.DataFrame([x for x in df_gravity.tags])
#df_gravity_new[['tags1','tags2']] = pd.DataFrame([x for x in df_gravity.tags])
df_gravity_new = df_gravity_new.drop('tags', axis=1)
df_gravity_new.head()

In [ ]:
list_tags = []
weight_tags = []
count_tags = 0

for i in range(len(df_gravity_new)):
    for j in range(3):
        if (df_gravity_new.iloc[i,j+2] in list_tags) == False:
            list_tags.append(df_gravity_new.iloc[i,j+2])
            weight_tags.append(j*0.5+1)
            count_tags = count_tags+1
        else:
            ind = list_tags.index(df_gravity_new.iloc[i,j+2])
            weight_tags[ind] = weight_tags[ind]+(j*0.5+1)

print (count_tags)
print (len(list_tags),len(weight_tags),list_tags[:5],weight_tags[:5])

### networkx

In [ ]:
import networkx as nx
from copy import deepcopy

G = nx.Graph()

pos = deepcopy(gravity)
count_gravity = len(gravity)
pos_tags = []
for i in range(count_tags):
    pos_tags.append((3*(i%10)-10,60-10*(i-i%10)/10+5*(i%2)))
    #pos_tags.append((5*(i-i%16)/16+2*(i%2)-20,(i%16)+30))
pos_points = []
[pos_points.append((df_visual.iloc[i,0],df_visual.iloc[i,1]))  for i in range(len(df_visual))]
pos = pos+pos_tags+pos_points
G.add_nodes_from(range(len(pos)))

fig = plt.figure(figsize=(12,12))
nx.draw_networkx_nodes(G,pos,nodelist=range(count_gravity),node_color='yellow',node_size=60)    
nx.draw_networkx_nodes(G,pos,nodelist=range(count_gravity,count_gravity+count_tags),node_color='orange',alpha=0.3,node_size=[50*i  for i in weight_tags])
#nx.draw_networkx_nodes(G,pos,nodelist=range(count_gravity+count_tags,len(pos)),node_color='green',node_size=10)

edges_premier = []
#edges = []
width_edges_premier = []
#width_edges = []
for i in range(len(df_gravity_new)):
    edges_premier.append((i,list_tags.index(df_gravity_new.tags1[i])+count_gravity))
    #edges.append((i,list_tags.index(df_gravity_new.tags2[i])+count_gravity))
    #edges.append((i,list_tags.index(df_gravity_new.tags3[i])+count_gravity))
    width_edges_premier.append(2.0)
    #width_edges.append(1.5)
    #width_edges.append(1.0)
    #edges.append((i,list_tags.index(df_gravity_new.tags4[i])+length_cluster))
    #edges.append((i,list_tags.index(df_gravity_new.tags5[i])+length_cluster))

    
ensemble_edges = []

for i in range(len(df_gravity_new)):
    ensemble_edges.append((i,list_tags.index(df_gravity_new.tags1[i])+count_gravity))
    ensemble_edges.append((i,list_tags.index(df_gravity_new.tags2[i])+count_gravity))
    ensemble_edges.append((i,list_tags.index(df_gravity_new.tags3[i])+count_gravity))    
    
    
#nx.draw_networkx_edges(G,pos,edgelist=edges,width=width_edges,edge_color='grey')
#nx.draw_networkx_edges(G,pos,edgelist=edges_premier,width=width_edges_premier,edge_color='grey')
G.add_edges_from(ensemble_edges)

edges_new = []
for i in range(len(df_visual)):
    edges_new.append((i+count_gravity+count_tags,df_visual.label[i]))
#nx.draw_networkx_edges(G,pos,edgelist=edges_new,width=1,alpha=0.5,edge_color='pink')    
G.add_edges_from(edges_new)


#print (G.nodes())
labels = {}
seuil = 3
for i in range(count_gravity):
    labels[i] = i
for i in range(count_tags):
    if weight_tags[i] > seuil:
        labels[len(gravity)+i] = list_tags[i]

arr_a = []        
edges = []
for a,b in ensemble_edges:
    if weight_tags[b-count_gravity] > seuil:
        edges.append((a,b))
        arr_a.append(a)

edges_brand = []
arr_brand = []
for a,b in edges_new:
    if (b in arr_a) != False:
        #print (a,b)
        edges_brand.append((a,b))
        arr_brand.append(a)
        
nx.draw_networkx_nodes(G,pos,nodelist=arr_brand,node_color='pink',node_size=10)
nx.draw_networkx_labels(G,pos,labels,front_size=1)
nx.draw_networkx_edges(G,pos,edgelist=edges,width=1,edge_color='grey',alpha=0.5)
nx.draw_networkx_edges(G,pos,edgelist=edges_brand,width=1,alpha=0.2,edge_color='red')    
G.add_edges_from(edges_brand)


plt.show()

### P4 : analyse notoriété d youtube brands

In [ ]:
list_youtube1 = y2.video_searched.values.tolist()
list_youtube2 = km_index.brand.values.tolist()

print (len(list_youtube1),len(list_youtube2))

arr_label = []
arr_alltags = []
for i in range(len(y2)):
    if (list_youtube1[i] in list_youtube2) == False:
        arr_label.append(12)
        arr_alltags.append(['Null'])
    else:
        ind = km_index['label'][list_youtube2.index(list_youtube1[i])]
        arr_label.append(km_index['label'][list_youtube2.index(list_youtube1[i])])
        arr_alltags.append(df_gravity['tags'][ind])
        
print (len(arr_label),arr_label[:5])        

In [ ]:
y2['label'] = pd.DataFrame(arr_label)
y2['sub_tags'] = pd.Series(arr_alltags)
y2.head()

In [ ]:
idx = y2.groupby(['label'])['score3'].transform(max) == y2['score3']
y2_view_sort = y2[idx]
y2_view_sort = y2_view_sort.reset_index()
y2_view_sort = y2_view_sort.drop('index',axis=1)
y2_view_sort.iloc[:10,:]

In [ ]:
yt_top10_brand = sorted(range(len(y2)), key=lambda k:y2.score2.values.tolist()[k],reverse=True)[:20]
for i in yt_top10_brand:
    print (y2.video_searched.values.tolist()[i],y2.score2.values.tolist()[i],y2.sub_tags[i])

In [ ]:
for i in range(len(y2_view_sort)):
    print (y2_view_sort.video_searched.values.tolist()[i],y2_view_sort.score2.values.tolist()[i],y2_view_sort.sub_tags[i])